# နမူနာ ၀၁: OpenAI SDK ကို အသုံးပြု၍ အမြန်စကားပြောခြင်း

ဒီ notebook က OpenAI SDK ကို Microsoft Foundry Local နဲ့အတူ အသုံးပြုပြီး အမြန်စကားပြောလုပ်ဆောင်ပုံကို ပြသထားပါတယ်။

## အကျဉ်းချုပ်

ဒီနမူနာမှာ ပြသထားတာတွေက:
- Foundry Local နဲ့အတူ OpenAI Python SDK ကို အသုံးပြုခြင်း
- Azure OpenAI နဲ့ Foundry Local configuration နှစ်ခုလုံးကို ကိုင်တွယ်ခြင်း
- အမှားများကို သင့်တော်သောနည်းလမ်းဖြင့် ကိုင်တွယ်ခြင်းနှင့် fallback strategy များကို အကောင်အထည်ဖော်ခြင်း
- FoundryLocalManager ကို အသုံးပြု၍ service ကို စီမံခန့်ခွဲခြင်း


## အခြေခံလိုအပ်ချက်များ

သေချာစွာရှိထားပါ:
1. Foundry Local ကို ထည့်သွင်းပြီး အလုပ်လုပ်နေသည်
2. မော်ဒယ်တစ်ခုကို load လုပ်ထားသည် (ဥပမာ `foundry model run phi-4-mini`)
3. လိုအပ်သော Python packages များကို ထည့်သွင်းပြီးဖြစ်သည်

### Dependencies များကို ထည့်သွင်းရန်


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## လိုအပ်သောစာကြည့်တိုက်များကိုတင်သွင်းပါ


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## ဖွဲ့စည်းမှု

သင့်ဖွဲ့စည်းမှုကို စတင်ပြင်ဆင်ပါ။ သင်အသုံးပြုနိုင်သည်မှာ:
1. **Azure OpenAI** (မိုဃ်းတိမ်အခြေပြု)
2. **Foundry Local** (ဒေသခံအတုယူခြင်း)

### ရွေးချယ်မှု ၁: Azure OpenAI ဖွဲ့စည်းမှု


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### အရွေးချယ်မှု ၂ - Foundry ဒေသခံဖွဲ့စည်းမှု


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## Client Initialization

သင့် configuration အရ OpenAI client ကို ဖန်တီးပါ:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## အခြေခံ Chat ဥပမာ

ရိုးရိုး chat အပြန်အလှန်ကို စမ်းကြည့်ရအောင်:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## အပြန်အလှန် စကားဝိုင်း

မော်ဒယ်ရဲ့ တုံ့ပြန်မှုကို ကြည့်ရှုရန် အမျိုးမျိုးသော အစပြုစကားများကို စမ်းကြည့်ပါ:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## အဆင့်မြင့် အသုံးပြုမှု: Streaming Responses

အကြမ်းမားသော အဖြေများအတွက် Streaming သည် ပိုမိုကောင်းမွန်သော အသုံးပြုသူအတွေ့အကြုံကို ပေးနိုင်ပါသည်။


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## အမှားကိုင်တွယ်ခြင်းနှင့် ရောဂါရှာဖွေခြင်း

ဝန်ဆောင်မှုအခြေအနေနှင့် ရရှိနိုင်သောမော်ဒယ်များကို စစ်ဆေးကြမယ်:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## အထူး Prompt စမ်းသပ်ခြင်း

အောက်ပါ cell ကို အသုံးပြု၍ သင့်ကိုယ်ပိုင် prompt များကို စမ်းသပ်ပါ:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## အကျဉ်းချုပ်

ဒီ notebook မှာ အောက်ပါအရာတွေကို ပြသထားပါတယ် -

1. **✅ Client Setup**: Foundry Local နဲ့ OpenAI SDK ကို ဘယ်လို configure လုပ်ရမလဲ
2. **✅ Basic Chat**: ရိုးရိုး request-response အပြန်အလှန်
3. **✅ Streaming**: အချိန်နှင့်တပြေးညီ response ထုတ်ပေးခြင်း
4. **✅ Error Handling**: အမှားများကို ချုပ်ကိုင်ပြီး အကဲဖြတ်နိုင်မှု
5. **✅ Service Health**: မော်ဒယ်ရရှိနိုင်မှုနဲ့ ဝန်ဆောင်မှုအခြေအနေ စစ်ဆေးခြင်း

### နောက်ထပ်အဆင့်များ

- **Sample 02**: Azure OpenAI ကို ထောက်ခံတဲ့ အဆင့်မြင့် SDK ပေါင်းစပ်မှု
- **Sample 04**: Chainlit chat application တည်ဆောက်ခြင်း
- **Sample 05**: Multi-agent orchestration systems
- **Sample 06**: Intelligent model routing

### Foundry Local ရဲ့ အဓိက အကျိုးကျေးဇူးများ

- 🔒 **Privacy**: ဒေတာတွေက သင့် device ထဲက မထွက်ဘူး
- ⚡ **Speed**: အချိန်နှင့်တပြေးညီ အမြန်ဆုံး inference
- 💰 **Cost**: API အသုံးပြုမှု ကုန်ကျစရိတ်မရှိ
- 🔌 **Offline**: အင်တာနက်မလိုအပ်ဘဲ အလုပ်လုပ်နိုင်
- 🛠️ **Compatibility**: OpenAI-compatible API
